In [1]:
from __future__ import print_function
import sys,os,datetime
import cv2 as cv
import cv2 as cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import sklearn.metrics as skm
import csv

In [2]:
print('openCV',cv.__version__)
print('numpy',np.__version__)

openCV 3.2.0
numpy 1.13.3


In [16]:
dirTrain1 = '../Data/train/train/level-1'
dirTrain2 = '../Data/train/train/level-2'
dirTrain3 = '../Data/train/train/level-3'
dirTrain  = dirTrain2
dirDev    = '../Data/dev/dev'
dirForSave = '../Work'

In [11]:
shape2D = (416,416)

In [17]:
def NNTrain(dirTrain, ends='.txt') :
    listTrain= os.listdir(dirTrain); len(listTrain); listTrain[:12]
    listTrainNN = []
    for ff in listTrain :
        if ff.lower().endswith(ends) : listTrainNN.append(ff.split('_')[0])
    return(listTrainNN)
listTrainNN = NNTrain(dirTrain)
len(listTrainNN), listTrainNN[:15]

(1495,
 ['1003',
  '1009',
  '100',
  '1012',
  '1018',
  '1021',
  '984',
  '994',
  '995',
  '997',
  '9',
  '952',
  '955',
  '956',
  '959'])

In [6]:
# Build statistics on masks
img1 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'.tif'),-1)
img2 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'_ref.tif'),-1)
img3 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'_mask.tif'),-1)
if 0 :
    for dirTrain in (dirTrain1,dirTrain2,dirTrain3) :
        listTrainNN = NNTrain(dirTrain)
        maskSize = []
        for NN in range(len(listTrainNN)) :
            #print(NN)
            img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
            img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)
            img3 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_mask.tif'),-1)
            maskSize.append(img3[img3>0.5].size)
            #img1      = cv.GaussianBlur(img1,(3,3),0)
            #img2      = cv.GaussianBlur(img2,(3,3),0)
        print(img1.shape,img2.shape,img3.shape, dirTrain)

        maskSize = np.array(maskSize)
        print(len(maskSize), len(maskSize[maskSize==0]), maskSize.mean(), np.median(maskSize), maskSize.std(), maskSize.mean()/(390*390)*100.0, float(maskSize.max())/(390*390)*100.0)
        plt.hist(maskSize,bins=150); plt.show()

In [7]:
def diffPicts (img1, img2) :
    diff = []
    for cc in range(img1.shape[2]) :
        ccc   = np.abs(img1[:,:,cc]-img2[:,:,cc])
        cccL2 = np.sqrt(np.square(ccc).mean())
        cccL1 = ccc.max()
        diff.append([cccL1,cccL2,ccc])
    diffL2 = np.zeros(ccc.shape)
    for L1,L2,ccc in diff : diffL2 += np.square(ccc)
    diffL2 = np.sqrt(diffL2)
    return(diff,diffL2)

def minDeltaRC (img1,img2, ranging=range(-5,6), printOK=False) :
    r0,c0 = (img2.shape[0]-img1.shape[0])/2, (img2.shape[1]-img1.shape[1])/2
    minL2 = [390*390*250.0,0,0]
    for dr in ranging :
        for dc in ranging :
            imgX2 = img2[r0+dr:r0+img1.shape[0]+dr,c0+dc:c0+img1.shape[1]+dc,:]
            diff, L2 = diffPicts(img1,imgX2)
            if (minL2[0]>L2.mean()) : minL2 = (L2.mean(),dr,dc,r0+dr,c0+dc,r0,c0)
            if printOK : print (dr,dc,'\t',minL2[0],'\t',L2.max(),L2.mean(),L2.std(),np.median(L2))
    return(minL2)

print('min=',minDeltaRC(img1,img2))

min= (10.974686533082986, 0, 0, 55, 55, 55, 55)


In [8]:
def pictNorm (img255, std=False) :
    imgF32 = np.float32(img255) 
    for cc in range(img255.shape[2]) :
        imgF32[:,:,cc] -= imgF32[:,:,cc].mean()
        if std : imgF32[:,:,cc] /= imgF32[:,:,cc].std()**2
        imgF32[:,:,cc] /= 127.0
    return(imgF32)

In [9]:
def computeORB (img, debug=False) :
    orb    = cv2.ORB_create();
    kp     = orb.detect(img,None)
    kp,des = orb.compute(img,kp)
    if debug :
        print (len(kp))
        print (len(des),len(des[0]),"\n",des[0],"\n",des[1])
    return kp,des

def computeMatches (des1,des2) :
    if True :
        bf      = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
        matches = bf.match(des1,des2);
        #????matches = bf.knnMatch(des1,des2,k=1);
    if False :
        index_params = dict(algorithm = 6, #cv2.FLANN_INDEX_LSH,
                            table_number = 6, 
                            key_size = 12,    
                            multi_probe_level = 1)
        search_params = dict(checks=100)
        flann   = cv2.FlannBasedMatcher(index_params,search_params)
        matchesI = flann.knnMatch(des1,des2,k=3);
        matches  = [];
        for mI in matchesI :
            for m in mI : matches.append(m);
    return matches    

def drawMatches(img1,kp1,des1,img2,kp2,des2, show=False, showNN=20):
    matches = computeMatches(des1,des2);
    matches = sorted(matches,key=lambda x:x.distance)
    if  show :
        imgM    = img1.copy()
        #imgM    = cv2.drawMatches(img1,kp1,img2,kp2,matches[:20],imgM,flags=4)
        imgM    = cv2.drawMatches(img1,kp1,img2,kp2,matches[:showNN],imgM,flags=2) #4)
        #imgM    = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,imgM,flags=2) #4)
        
        plt.figure(figsize=(15,15)); plt.imshow(imgM), plt.show()
    return(matches)


In [10]:
len(listTrainNN)

1495

In [13]:
print(datetime.datetime.now())

for ii,dirTrain in enumerate((dirTrain1,dirTrain2,dirTrain3)) :
    
    #if ii==1 : break
    
    listTrainNN = NNTrain(dirTrain)

    X1 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    YY = np.zeros((len(listTrainNN),)+shape2D+(1,),dtype=np.float32)
    print(datetime.datetime.now(),X1.shape, X2.shape,YY.shape,dirTrain)
    
    for NN in range(len(listTrainNN)) :
        img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
        img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)
        img3 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_mask.tif'),-1); img3 = img3.reshape(img3.shape+(1,))



        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2 = cv2.GaussianBlur(img2,(3,3),0)

        if 0 :
            img1 = cv2.boxFilter(img1,-1,(5,5))
            img2 = cv2.boxFilter(img2,-1,(5,5))


        if 1 :
            img1  = pictNorm(img1)
            img2  = pictNorm(img2)

        rcL2,_,_,r0,c0,ro,co  = minDeltaRC(img1,img2,ranging=range(-10,10))
        #img2X = img2[r0:r0+img1.shape[0],c0:c0+img1.shape[1]]
        img2X = np.array(img2[ro:(ro+img1.shape[0]),co:co+img1.shape[1]])
        
        #print(img2[ro:ro+img1.shape[0],co:co+img1.shape[1]].shape)
        #print(img2.shape,img2X.shape,ro,ro+img1.shape[0],co,co+img1.shape[1])
        
        if NN%100==0 : print(datetime.datetime.now(),NN,rcL2,ro,co,r0,c0)

        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2X = cv2.GaussianBlur(img2X,(3,3),0)

        if 0 :
            img1  = cv2.boxFilter(img1, -1,(5,5))
            img2X = cv2.boxFilter(img2X,-1,(5,5))

        X1[NN,13:13+390,13:13+390,:], X2[NN,13:13+390,13:13+390,:], YY[NN,13:13+390,13:13+390,:] = img1, img2X, img3


    if 1 :
        np.save(os.path.join(dirForSave,'NN-'+str(ii)+'S.npy'),np.array(listTrainNN))
        np.save(os.path.join(dirForSave,'X1-'+str(ii)+'S.npy'),X1)
        np.save(os.path.join(dirForSave,'X2-'+str(ii)+'S.npy'),X2)
        np.save(os.path.join(dirForSave,'YY-'+str(ii)+'S.npy'),YY)
        print(datetime.datetime.now(),'Save:',X1.shape, X2.shape,YY.shape,dirTrain)
        
    del X1,X2,YY
        
print(datetime.datetime.now())

2018-02-15 01:48:13.259479
2018-02-15 01:48:13.317090 (751, 416, 416, 3) (751, 416, 416, 3) (751, 416, 416, 1) ../Data/train/train/level-1
2018-02-15 01:48:14.979299 0 0.126156769139 55 55 56 57
2018-02-15 01:51:17.021204 100 0.107539980338 55 55 56 55
2018-02-15 01:54:23.157235 200 0.112199069454 55 55 59 50
2018-02-15 01:57:33.826924 300 0.0941469409104 55 55 55 50
2018-02-15 02:00:25.293811 400 0.097671324921 55 55 46 63
2018-02-15 02:02:54.809664 500 0.0944290245647 55 55 54 56
2018-02-15 02:05:17.560839 600 0.134311490241 55 55 55 55
2018-02-15 02:07:37.915710 700 0.216116584654 55 55 55 52
2018-02-15 02:09:22.797033 Save: (751, 416, 416, 3) (751, 416, 416, 3) (751, 416, 416, 1) ../Data/train/train/level-1
2018-02-15 02:09:23.224054 (1495, 416, 416, 3) (1495, 416, 416, 3) (1495, 416, 416, 1) ../Data/train/train/level-2
2018-02-15 02:09:24.457794 0 0.157420539797 55 55 55 55
2018-02-15 02:11:49.620565 100 0.174736018623 55 55 55 55
2018-02-15 02:14:11.583380 200 0.0686089521011 55 

In [26]:
if 0 :
    np.save(os.path.join(dirForSave,'X1.npy'),X1)
    np.save(os.path.join(dirForSave,'X2.npy'),X2)
    np.save(os.path.join(dirForSave,'YY.npy'),YY)
    
if 0 :
    ii = 0
    X1 = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'.npy'))
    X2 = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'.npy'))
    YY = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
    print(datetime.datetime.now(),'Load: ',ii,X1.shape, X2.shape,YY.shape)
    
if 0 :
    iil = []
    for ii in (0,1,2) :
        YY = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        iil.append(len(YY))
    iis = int(np.array(iil).sum())
    
    X1 = np.zeros((iis,)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((iis,)+shape2D+(3,),dtype=np.float32)
    YY = np.zeros((iis,)+shape2D+(1,),dtype=np.float32)
    
    iis = 0
    
    for ii in (0,1,2) :
        
        T1  = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'.npy'))
        T2  = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'.npy'))
        TT  = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        
        X1[iis:iis+len(TT)], X2[iis:iis+len(TT)], YY[iis:iis+len(TT)] = T1, T2, TT
        
        iis+= len(TT)
        
        del TT,T1, T2
        
    print(datetime.datetime.now(),'Load: ', X1.shape, X2.shape, YY.shape)
    

In [19]:
# Build Devs arrays
print(datetime.datetime.now())

for ii,dirTrain in enumerate((os.path.join(dirDev,'level-1'),
                              os.path.join(dirDev,'level-2'),
                              os.path.join(dirDev,'level-3'))) :
    
    #if ii==1 : break
    
    listTrainNN = NNTrain(dirTrain,ends='_ref.tif')

    X1 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    ##YY = np.zeros((len(listTrainNN),)+shape2D+(1,),dtype=np.float32)
    print(datetime.datetime.now(),X1.shape, X2.shape,dirTrain)
    
    for NN in range(len(listTrainNN)) :
        img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
        img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)



        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2 = cv2.GaussianBlur(img2,(3,3),0)

        if 0 :
            img1 = cv2.boxFilter(img1,-1,(5,5))
            img2 = cv2.boxFilter(img2,-1,(5,5))


        if 1 :
            img1  = pictNorm(img1)
            img2  = pictNorm(img2)

        rcL2,_,_,r0,c0,ro,co  = minDeltaRC(img1,img2,ranging=range(-10,10))
        #img2X = img2[r0:r0+img1.shape[0],c0:c0+img1.shape[1]]
        img2X = np.array(img2[ro:(ro+img1.shape[0]),co:co+img1.shape[1]])
        
        #print(img2[ro:ro+img1.shape[0],co:co+img1.shape[1]].shape)
        #print(img2.shape,img2X.shape,ro,ro+img1.shape[0],co,co+img1.shape[1])
        
        if NN%100==0 : print(datetime.datetime.now(),NN,rcL2,ro,co,r0,c0)

        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2X = cv2.GaussianBlur(img2X,(3,3),0)

        if 0 :
            img1  = cv2.boxFilter(img1, -1,(5,5))
            img2X = cv2.boxFilter(img2X,-1,(5,5))

        X1[NN,13:13+390,13:13+390,:], X2[NN,13:13+390,13:13+390,:] = img1, img2X


    if 1 :
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-NN-S.npy'),np.array(listTrainNN))
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-X1-S.npy'),X1)
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-X2-S.npy'),X2)
        print(datetime.datetime.now(),'Save:',X1.shape, X2.shape,dirTrain)
        
    del X1,X2
        
print(datetime.datetime.now())

2018-02-15 03:47:08.891797
2018-02-15 03:47:08.894567 (245, 416, 416, 3) (245, 416, 416, 3) ../Data/dev/dev/level-1
2018-02-15 03:47:10.014160 0 0.105747467839 55 55 54 55
2018-02-15 03:48:56.798208 100 0.0915711377855 55 55 56 56
2018-02-15 03:52:13.362848 200 0.131872531383 55 55 62 55
2018-02-15 03:53:11.170329 Save: (245, 416, 416, 3) (245, 416, 416, 3) ../Data/dev/dev/level-1
2018-02-15 03:53:11.294400 (510, 416, 416, 3) (510, 416, 416, 3) ../Data/dev/dev/level-2
2018-02-15 03:53:12.385335 0 0.126518101886 55 55 54 55
2018-02-15 03:55:08.236691 100 0.120504920486 55 55 54 55
2018-02-15 03:56:58.435156 200 0.196706252378 55 55 54 56
2018-02-15 03:58:46.494024 300 0.173141766596 55 55 55 55
2018-02-15 04:00:32.720194 400 0.100078645487 55 55 54 54
2018-02-15 04:02:26.660457 500 0.0835766008211 55 55 55 55
2018-02-15 04:02:48.195404 Save: (510, 416, 416, 3) (510, 416, 416, 3) ../Data/dev/dev/level-2
2018-02-15 04:02:49.148648 (650, 416, 416, 3) (650, 416, 416, 3) ../Data/dev/dev/leve